In [80]:
from langchain_ollama.llms import OllamaLLM
from datasets import load_dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
import pandas as pd
import numpy as np

import json

In [81]:
data = load_dataset("SaguaroCapital/sentiment-analysis-in-commodity-market-gold")

In [82]:
print(len(data['test']['News']))
print(len(data['test']['Price Sentiment']))
print(np.unique(data['test']['Price Sentiment']))
data['test']['News'][0:7]

2114
2114
['negative' 'neutral' 'none' 'positive']


['Gold / Silver / Copper futures - weekly outlook: February 27 - March 3',
 'gold to be a safe-haven again; sell crude on rally: barratt',
 'feb. gold settles at $1,097,90/oz on comex, down 0.9% for the session',
 'dec gold rises 30c to $443.40/oz in morning ny trade',
 'Gold holds modest losses after Chicago PMI miss',
 'December gold $4.90, or 0.4%, lower at $1,313.20/oz.',
 'gold prices gain in asia on technical rebound, boj ahead']

In [83]:
test = data['test']['Price Sentiment'][:400]
data_used = data['test']['News'][:400]

In [84]:
# the data contains 'none' and 'neutral' - changing none to neutral
test = ['neutral' if x == 'none' else x for x in test]
np.unique(test)

array(['negative', 'neutral', 'positive'], dtype='<U8')

In [85]:
sa_llm = OllamaLLM(
    model="mattarad/llama3.2-3b-instruct-mqc-sa",  # Updated to 1b model
    base_url="http://localhost:11434",  # Add explicit base URL
    temperature=0.25
)

In [86]:
predictions = []
for index, sent in enumerate(data_used):
    sentiment = sa_llm.invoke(sent)
    res = json.loads(sentiment)
    label = res["label"]
    sentiment = res["sentiment"]
    predictions.append(label)
    if index % 50 == 0:
        print(f"{index + 1} / {len(data_used)}")
        print(predictions)

print(predictions)

1 / 400
['neutral']
51 / 400
['neutral', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'neutral', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'positive', 'positive', 'positive', 'neutral', 'neutral', 'negative', 'negative', 'positive', 'neutral', 'negative', 'positive', 'positive', 'negative', 'positive', 'positive', 'negative', 'negative', 'positive', 'negative', 'negative', 'neutral', 'negative', 'negative', 'negative', 'positive', 'positive', 'negative', 'positive', 'positive', 'positive', 'negative', 'negative', 'neutral', 'negative', 'negative', 'negative', 'negative']
101 / 400
['neutral', 'negative', 'negative', 'positive', 'negative', 'negative', 'positive', 'neutral', 'positive', 'positive', 'positive', 'negative', 'positive', 'negative', 'positive', 'positive', 'positive', 'neutral', 'neutral', 'negative', 'negative', 'positive', 'neutral', 'negative', 'positive', 'positive', 'negative', 'positive', 'positive', 'negative',

In [87]:
print(test[:2])
data_used_df = pd.DataFrame(data_used, columns=['sentence'])
test_df = pd.DataFrame(test, columns=['test'])
pred_df = pd.DataFrame(predictions, columns=['pred'])
combined_df = pd.concat([data_used_df, test_df, pred_df], axis=1)
combined_df.head()


['neutral', 'neutral']


,sentence,test,pred
0,Gold / Silver / Copper futures - weekly outlook: February 27 - March 3,neutral,neutral
1,gold to be a safe-haven again; sell crude on rally: barratt,neutral,negative
2,"feb. gold settles at $1,097,90/oz on comex, down 0.9% for the session",negative,negative
3,dec gold rises 30c to $443.40/oz in morning ny trade,positive,positive
4,Gold holds modest losses after Chicago PMI miss,negative,negative


In [88]:
pd.options.display.max_colwidth = None
new_df = combined_df[combined_df['test'] != combined_df['pred']]
new_df.head()

,sentence,test,pred
1,gold to be a safe-haven again; sell crude on rally: barratt,neutral,negative
7,"Buy gold if it dips to $1,245-48/oz",negative,neutral
8,"peter schiff sheds no tears as a trump rally hammers gold, dow nears 20,000",neutral,positive
15,Gold Prices Hold onto Gains Ahead of FOMC Statement,neutral,positive
20,"why apple's $10k, solid-gold watch isn't dazzling metals experts",neutral,negative


#### the results aren't as good as they should be since the dataset is scored on ONLY the gold phrase in the training line and not the full line.
#### e.g. the line below is ranked as 'none' by the dataset, but our model outputs negative due to the 'sell crude on rally'  
***'gold to be a safe-haven again; sell crude on rally: barratt'***  

#### the dataset also has a 'none' option, which we changed neutral that could be throwing off the analysis

In [89]:
accuracy = accuracy_score(test, predictions)
precision = precision_score(test, predictions, average='weighted')
recall = recall_score(test, predictions, average='weighted')
f1 = f1_score(test, predictions, average='weighted')

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1-score:", f1)

Accuracy: 0.735
Precision: 0.7185923280423281
Recall: 0.735
F1-score: 0.7228290142563643
